In [ ]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

In [ ]:
#url = "https://raw.githubusercontent.com/adam-datasci/RiskAnalytics/main/Data/american_bankruptcy.csv?"
url = "Data/american_bankruptcy_normalized.csv"
df = pd.read_csv(url)

In [ ]:
features = ['wc_to_ta_pct','re_to_ta_pct','ebit_to_ta_pct','mv_to_tl_pct','tr_to_ta_pct','default']

In [ ]:
default_periods = df[df['default'] == 1]
non_default_periods = df[df['default'] == 0]
random_sample = non_default_periods.sample(n=609, random_state=1)
df_balanced = pd.concat([default_periods, random_sample])
df_balanced_features = df_balanced[features]
df_balanced_features_melt = df_balanced_features.melt(id_vars='default', value_vars=features[:-1],var_name='ratio_name',value_name='ratio_value')